<a href="https://colab.research.google.com/github/student-monika/Marvel_tasks_Level_2/blob/main/ID3_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import pandas as pd

# Function to calculate entropy
def calculate_entropy(data):
    labels = data.iloc[:, -1]
    total_count = len(labels)
    label_counts = labels.value_counts()
    entropy = -sum((count / total_count) * math.log2(count / total_count) for count in label_counts)
    return entropy

# Function to calculate information gain
def calculate_information_gain(data, feature):
    total_entropy = calculate_entropy(data)
    total_count = len(data)
    values = data[feature].unique()
    weighted_entropy = sum(
        (len(subset) / total_count) * calculate_entropy(subset)
        for value in values
        for subset in [data[data[feature] == value]]
    )
    return total_entropy - weighted_entropy

# Recursive ID3 algorithm
def id3(data, features, tree=None):
    # If all instances have the same class, return that class
    if len(data.iloc[:, -1].unique()) == 1:
        return data.iloc[0, -1]

    # If no features are left, return the majority class
    if not features:
        return data.iloc[:, -1].mode()[0]

    # Select the feature with the highest information gain
    gains = {feature: calculate_information_gain(data, feature) for feature in features}
    best_feature = max(gains, key=gains.get)

    # Initialize the tree structure
    tree = {best_feature: {}}

    # Split the dataset on the best feature
    for value in data[best_feature].unique():
        subset = data[data[best_feature] == value]
        subtree = id3(subset.drop(columns=[best_feature]), [f for f in features if f != best_feature])
        tree[best_feature][value] = subtree

    return tree

# Function to make a prediction for a single test instance
def predict(tree, instance):
    if not isinstance(tree, dict):  # Leaf node
        return tree
    root_feature = next(iter(tree))
    feature_value = instance[root_feature]
    if feature_value in tree[root_feature]:  # Move down the tree
        return predict(tree[root_feature][feature_value], instance)
    else:
        return None  # Handle unseen feature values

# Function to calculate accuracy
def calculate_accuracy(tree, test_data):
    correct_predictions = 0
    for _, row in test_data.iterrows():
        prediction = predict(tree, row)
        if prediction == row.iloc[-1]:
            correct_predictions += 1
    accuracy = correct_predictions / len(test_data)
    return accuracy

# Example dataset
data = pd.DataFrame({
    "Outlook": ["Sunny", "Sunny", "Overcast", "Rain", "Rain", "Rain", "Overcast", "Sunny", "Sunny", "Rain", "Sunny", "Overcast", "Overcast", "Rain"],
    "Temperature": ["Hot", "Hot", "Hot", "Mild", "Cool", "Cool", "Cool", "Mild", "Cool", "Mild", "Mild", "Mild", "Hot", "Mild"],
    "Humidity": ["High", "High", "High", "High", "Normal", "Normal", "Normal", "High", "Normal", "Normal", "Normal", "High", "Normal", "High"],
    "Windy": [False, True, False, False, False, True, True, False, False, False, True, True, False, True],
    "Play": ["No", "No", "Yes", "Yes", "Yes", "No", "Yes", "No", "Yes", "Yes", "Yes", "Yes", "Yes", "No"]
})

# Split dataset into training and testing sets (e.g., 70% train, 30% test)
train_data = data.iloc[:10]  # First 10 rows for training
test_data = data.iloc[10:]   # Remaining rows for testing

# Build the decision tree
features = ["Outlook", "Temperature", "Humidity", "Windy"]
decision_tree = id3(train_data, features)

# Calculate accuracy on the test dataset
accuracy = calculate_accuracy(decision_tree, test_data)

print("Decision Tree:", decision_tree)
print("Accuracy on test dataset:", accuracy)


Decision Tree: {'Outlook': {'Sunny': {'Temperature': {'Hot': 'No', 'Mild': 'No', 'Cool': 'Yes'}}, 'Overcast': 'Yes', 'Rain': {'Windy': {False: 'Yes', True: 'No'}}}}
Accuracy on test dataset: 0.75
